In [1]:
import pybullet as p
import pybullet_data
import random
import matplotlib.pyplot as plt
import time
import numpy as np
import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import PPO # Or A2C, DQN for discrete actions
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv
import time
from stable_baselines3.common.callbacks import ProgressBarCallback

pybullet build time: Jan 29 2025 23:16:28


In [11]:

class MazeCarEnv(gym.Env):
    metadata = {'render_modes': ['human'], "render_fps": 500}

    def __init__(self, render_mode=None):
        super().__init__()

        # --- Define Action Space ---
        self.action_space = spaces.Discrete(3)

        # --- Define Observation Space ---
        low = np.array([-6, -6, -np.pi, -6, -6], dtype=np.float32)
        high = np.array([6, 6, np.pi, 6, 6], dtype=np.float32)
        self.observation_space = spaces.Dict({
            "state": spaces.Box(
                low=np.array([-6, -6, -np.pi, -6, -6], dtype=np.float32),
                high=np.array([6, 6, np.pi, 6, 6], dtype=np.float32),
                dtype=np.float32
            ),
            "camera": spaces.Box(
                low=0,
                high=255,
                shape=(64, 64, 4),  # RGBA image from PyBullet
                dtype=np.uint8
            )
        })

        # --- PyBullet Setup ---
        self.render_mode = render_mode
        self.client = p.connect(p.DIRECT if render_mode is None else p.GUI)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81, physicsClientId=self.client)

        self.planeId = p.loadURDF("plane.urdf", physicsClientId=self.client)

        self.goal_area_1 = np.array([4.5, 4.5])   # Upper goal
        self.goal_area_2 = np.array([4.5, -4.5])  # Lower goal
        self.goal_radius = 0.5
        self.target_goal_pos = None  # Will be set in reset()
        self.correct_goal_index = None  # Random selection of the target goal

        self.goal_spheres = []

        # Car (robot) loading
        self.start_pos = [-4.5, 0.0, 0.1]  # Start near the left side of the maze
        self.start_orn = p.getQuaternionFromEuler([0, 0, 0])
        self.carId = p.loadURDF("urdf/simple_two_wheel_car.urdf",
                                self.start_pos, self.start_orn,
                                physicsClientId=self.client)
        
        self.left_wheel_joint_index = 1
        self.right_wheel_joint_index = 0

        self.step_counter = 0
        self.max_steps_per_episode = 200

        self.action_repeat = 50

    def _get_obs(self):
        pos, orn_quat = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        euler = p.getEulerFromQuaternion(orn_quat)
        yaw = euler[2]

        camera_image = self._get_camera_image()

        return {
            "state": np.array([pos[0], pos[1], yaw, self.target_goal_pos[0], self.target_goal_pos[1]], dtype=np.float32),
            "camera": camera_image
        }

    def _get_info(self):
        car_pos, _ = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        dist_goal1 = np.linalg.norm(np.array(car_pos[:2]) - self.goal_area_1)
        dist_goal2 = np.linalg.norm(np.array(car_pos[:2]) - self.goal_area_2)

        return {
            "distance_goal1": dist_goal1,
            "distance_goal2": dist_goal2,
            "target_goal_index": self.correct_goal_index
        }

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.step_counter = 0

        # Randomly generate a maze
        maze = self.generate_maze(21, 21)  # You can adjust the size of the maze here
        self.create_maze_walls(maze)

        # Reset the car's position
        start_x = self.start_pos[0] + self.np_random.uniform(-0.3, 0.3)
        start_y = self.start_pos[1] + self.np_random.uniform(-0.3, 0.3)
        start_yaw = self.np_random.uniform(-np.pi/6, np.pi/6)
        start_orn = p.getQuaternionFromEuler([0, 0, start_yaw])
        p.resetBasePositionAndOrientation(self.carId, [start_x, start_y, self.start_pos[2]], start_orn, physicsClientId=self.client)
        p.resetBaseVelocity(self.carId,
                            linearVelocity=[0, 0, 0],
                            angularVelocity=[0, 0, 0],
                            physicsClientId=self.client)

        # Remove all existing multibodies
        multibodies = [p.getBodyUniqueId(i) for i in range(p.getNumBodies(physicsClientId=self.client))]
        for body_id in multibodies:
            p.removeBody(body_id, physicsClientId=self.client)
        self.goal_spheres.clear()

        # Randomly choose the "correct" goal
        self.correct_goal_index = self.np_random.integers(0, 2)
        
        if self.correct_goal_index == 0:
            self.target_goal_pos = self.goal_area_1
        else:
            self.target_goal_pos = self.goal_area_2

        observation = self._get_obs()
        info = self._get_info()

        return observation, info

    def step(self, action):
        if action == 0:
            left_vel, right_vel = 10.0, 10.0
        elif action == 1:
            left_vel, right_vel = 2.0, 10.0
        elif action == 2:
            left_vel, right_vel = 10.0, 2.0

        p.setJointMotorControl2(
            bodyUniqueId=self.carId,
            jointIndex=self.left_wheel_joint_index,
            controlMode=p.VELOCITY_CONTROL,
            targetVelocity=left_vel,
            force=20.0
        )
        p.setJointMotorControl2(
            bodyUniqueId=self.carId,
            jointIndex=self.right_wheel_joint_index,
            controlMode=p.VELOCITY_CONTROL,
            targetVelocity=right_vel,
            force=20.0
        )

        # Step through the simulation multiple times to simulate the action
        for _ in range(self.action_repeat):
            p.stepSimulation()

        self.step_counter += 1

        # Observation + reward + termination check
        observation = self._get_obs()
        info = self._get_info()

        terminated = False
        reward = -0.005

        car_pos, _ = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        curr_dist_to_goal = np.linalg.norm(np.array(car_pos[:2]) - self.target_goal_pos)
        reward = 0.1 * (self.prev_dist_to_goal - curr_dist_to_goal)
        self.prev_dist_to_goal = curr_dist_to_goal

        in_goal_1 = np.linalg.norm(car_pos[:2] - self.goal_area_1) < self.goal_radius
        in_goal_2 = np.linalg.norm(car_pos[:2] - self.goal_area_2) < self.goal_radius

        if in_goal_1:
            if self.correct_goal_index == 0:
                reward = 20.0
                terminated = True
            else:
                reward = -15.0
                terminated = True
        elif in_goal_2:
            if self.correct_goal_index == 1:
                reward = 20.0
                terminated = True
            else:
                reward = -15.0
                terminated = True

        contacts = p.getContactPoints(bodyA=self.carId, bodyB=self.mazeId, physicsClientId=self.client)
        if len(contacts) > 0:
            reward -= 1.0  # Penalty for collision with the maze wall
            terminated = True 

        truncated = (self.step_counter >= self.max_steps_per_episode)

        return observation, reward, terminated, truncated, info

    def generate_maze(self, width, height):
        # Create an empty maze
        maze = np.zeros((height, width), dtype=int)

        # Directions (Up, Down, Right, Left)
        directions = [(-1, 0), (1, 0), (0, 1), (0, -1)]

        def is_valid_move(x, y):
            return 0 <= x < height and 0 <= y < width and maze[x][y] == 0

        def carve(x, y):
            maze[x][y] = 1
            random.shuffle(directions)

            for dx, dy in directions:
                nx, ny = x + dx * 2, y + dy * 2
                if is_valid_move(nx, ny):
                    maze[x + dx][y + dy] = 1
                    carve(nx, ny)

        # Start carving from a random position
        start_x = random.randint(0, (height - 1) // 2) * 2 + 1  # Ensures odd indices for carving
        start_y = random.randint(0, (width - 1) // 2) * 2 + 1   # Ensures odd indices for carving
        carve(start_x, start_y)

        # Ensure start and end are open (avoid out-of-bounds errors)
        maze[start_x][start_y] = 0
        maze[height - 2][width - 2] = 0  # Set bottom-right corner as the goal

        return maze


    # def create_maze_walls(self, maze):
    #     wall_urdf = "urdf/wall.urdf"  # Path to a simple wall URDF
    #     maze_height, maze_width = maze.shape
    #     wall_objects = []

    #     for row in range(maze_height):
    #         for col in range(maze_width):
    #             if maze[row][col] == 1:  # Wall detected
    #                 wall_pos = [col * 1.0, row * 1.0, 0.5]
    #                 wall_id = p.loadURDF(wall_urdf, wall_pos, useFixedBase=True)
    #                 wall_objects.append(wall_id)

    #     self.mazeId = wall_objects

    def create_maze_walls(self, maze):
        wall_urdf = "urdf/wall.urdf"  # Path to the wall URDF

        # Print the URDF path to ensure it's correct
        print(f"Loading wall URDF from path: {wall_urdf}")

        maze_height, maze_width = maze.shape
        wall_objects = []

        for row in range(maze_height):
            for col in range(maze_width):
                if maze[row][col] == 1:  # Wall detected
                    wall_pos = [col * 1.0, row * 1.0, 0.5]  # Position of the wall (scaled by cell size)
                    try:
                        # Attempt to load the URDF
                        wall_id = p.loadURDF(wall_urdf, wall_pos, useFixedBase=True)
                        wall_objects.append(wall_id)
                    except Exception as e:
                        print(f"Failed to load URDF at position {wall_pos}: {e}")

        self.mazeId = wall_objects


    def render(self):
        # In PyBullet, rendering happens automatically in GUI mode
        pass

    def close(self):
        p.disconnect(physicsClientId=self.client)

    def _get_camera_image(self):
        car_pos, car_orn = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        car_euler = p.getEulerFromQuaternion(car_orn)

        camera_distance = 0.1
        camera_height = 0.2
        camera_yaw = car_euler[2] * 180 / np.pi
        target_pos = [
            car_pos[0] + camera_distance * np.cos(car_euler[2]),
            car_pos[1] + camera_distance * np.sin(car_euler[2]),
            car_pos[2] + camera_height
        ]

        width, height, rgba, _, _ = p.getCameraImage(
            width=64,
            height=64,
            viewMatrix=p.computeViewMatrix(
                cameraEyePosition=[car_pos[0], car_pos[1], car_pos[2] + camera_height],
                cameraTargetPosition=target_pos,
                cameraUpVector=[0, 0, 1]
            ),
            projectionMatrix=p.computeProjectionMatrixFOV(
                fov=70,
                aspect=1.0,
                nearVal=0.01,
                farVal=10.0
            ),
            physicsClientId=self.client
        )

        rgba_image = np.reshape(rgba, (height, width, 4)).astype(np.uint8)
        return rgba_image

In [3]:
# env = MazeCarEnv(render_mode=None) 
env = MazeCarEnv(render_mode="human")

p.resetDebugVisualizerCamera(cameraDistance=9, cameraYaw=-92, cameraPitch=-85, cameraTargetPosition=[0, 0, 0])

try:
    check_env(env)
    print("Environment check passed!")
except Exception as e:
    print(f"Environment check failed: {e}")
    env.close()
    exit()

model = PPO("MultiInputPolicy", 
            env, 
            ent_coef=0.05,
            # verbose=1, 
            tensorboard_log="./ppo_mazecar_tensorboard/",
            batch_size=128, # after n_steps the data is split into batches
            learning_rate=0.0003,
            n_steps=512,   # number of steps collected before a training
            device="cuda"
)


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (RPL-P)
GL_VERSION=4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
Vendor = Intel
Renderer = Mesa Intel(R) Graphics (RPL-P)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
Environment chec

In [4]:
# eval_env = MazeCarEnv(render_mode='human')
# eval_freq = 1000 
# eval_callback = EvalCallback(eval_env, eval_freq, verbose=1)

model.learn(total_timesteps=5000, progress_bar=True)

model.save("ppo_mazecar_model_2")
env.close()
# check_env.close()

Output()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


In [ ]:
# --- To evaluate ---
eval_env = MazeCarEnv(render_mode="human")  # Render during evaluation
model = PPO.load("ppo_mazecar_model_2_2", env=eval_env)

# Set the camera to the same position as for training
p.resetDebugVisualizerCamera(cameraDistance=9, cameraYaw=-92, cameraPitch=-85, cameraTargetPosition=[0, 0, 0])


print(".......................")
obs, info = eval_env.reset()
print(".......................")

for _ in range(500):  # Max steps for evaluation
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)
    time.sleep(1./120.0)  # Adjust sleep time for rendering speed
    if terminated or truncated:
        print(f"Eval episode finished, Reached goal: {info.get('target_goal_index')}, Reward: {reward}")
        obs, info = eval_env.reset()  # Reset for next evaluation episode

eval_env.close()

In [ ]:
# Load the previously saved model
model = PPO.load("ppo_mazecar_model_2", env=env)

# Continue training the model
model.learn(total_timesteps=50000, progress_bar=True)

# Save the updated model
model.save("ppo_mazecar_model_2_2")

Output()

KeyboardInterrupt: 

X connection to :0 broken (explicit kill or server shutdown).


: 

In [19]:
env.close()

check_env.close()

# car_pos, _ = p.getBasePositionAndOrientation(eval_env.carId, physicsClientId=eval_env.client)
# p.resetDebugVisualizerCamera(cameraDistance=3, cameraYaw=50, cameraPitch=-35, cameraTargetPosition=car_pos)


NameError: name 'env' is not defined

In [10]:
eval_env.close()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


In [ ]:
# ~/masterthesis/mt_start$   tensorboard --logdir=./ppo_mazecar_tensorboard/

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.get_device_name(0))  # Prints the name of the GPU

True
NVIDIA GeForce RTX 4060 Laptop GPU
